### Some sources taken code idea from

https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb

https://github.com/PlayingNumbers/ds_salary_proj/blob/master/glassdoor_scraper.py

https://stackoverflow.com/questions/27270357/closing-a-generic-pop-up-in-selenium

https://developer.chrome.com/docs/chromedriver/
https://www.glassdoor.co.in/robots.txt

https://mersakarya.medium.com/selenium-tutorial-scraping-glassdoor-com-in-10-minutes-3d0915c6d905

https://www.selenium.dev/

In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import pandas as pd

In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


## The code that did the scraping 

In [ ]:
options = Options()
#options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options = options)

driver.set_window_size(1120,1000)

url = "https://www.glassdoor.co.in/Job/" + "data-science-jobs" + "-SRCH_KO0,12.htm"
driver.get(url) #opens up this link in new chrome window
jobs = []
#num_jobs = 1


if(driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb')):
    driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb').click()
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    
# job_buttons = driver.find_elements(By.CLASS_NAME,"JobsList_jobListItem__wjTHv")

# companies = driver.find_elements(By.CLASS_NAME,'EmployerProfile_compactEmployerName__LE242')#.text
# descriptions = driver.find_elements(By.CLASS_NAME,"JobCard_jobDescriptionSnippet__yWW8q")#.text

#show_more = driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb')
#if(show_more):
#    show_more.click()
for j in range(60):
    
    if(driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb')):
        driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb').click()
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    job_buttons = driver.find_elements(By.CLASS_NAME,"JobsList_jobListItem__wjTHv")

    companies = driver.find_elements(By.CLASS_NAME,'EmployerProfile_compactEmployerName__LE242')#.text
    descriptions = driver.find_elements(By.CLASS_NAME,"JobCard_jobDescriptionSnippet__yWW8q")#.text
    salaries = driver.find_elements(By.CLASS_NAME,'JobCard_salaryEstimate__arV5J')#.text

        
    #driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
        
    for i,button in enumerate(job_buttons):
        #     show_more = driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb')
        #     if(show_more):
        #         show_more.click()
        #         if(driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb')):
        #             driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb').click()
        #             driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
        
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
        try:
            button.click()
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
        except:
            continue
        time.sleep(2)
    

        company_name = companies[i].text
        location = driver.find_element(By.CLASS_NAME,'JobDetails_location__mSg5h').text
        #job_title = driver.find_element(By.CLASS_NAME,'heading_Heading__BqX5J').text
        job_title = driver.find_element(By.XPATH, '//div[@class="JobDetails_jobDetailsHeader__Hd9M3"]//h1').text
        job_description = descriptions[i].text
        #job_description = driver.find_element(By.CLASS_NAME,'JobDetails_jobDetailsSection__y_xa6').text
        #job_description = driver.find_element(By.CLASS_NAME,'JobDetails_jobDescription__uW_fk').text
        
        try:
            salary_estimate = salaries[i].text
        except:
            salary_estimate = -1
            
        try:
            other_details = driver.find_element(By.CLASS_NAME, "JobDetails_companyOverviewGrid__3t6b4").text
        except:
            other_details = None
            
        try:
            rating = driver.find_element(By.CLASS_NAME, "RatingHeadline_sectionRatingScoreLeft__di1of").text
        except:
            rating = None
    
        jobs.append({'company_name':company_name, 'location':location, 'job_title':job_title, 
                     'job_description':job_description, 'salary_estimate':salary_estimate, 
                     'other_details' : other_details, 'rating':rating})
    
jobs

In [5]:
df = pd.DataFrame(jobs)

In [8]:
len(jobs)

1193

In [9]:
df.to_csv("jobs.csv")

## Chatgpt's correction code to avoid duplicates 
But this too couldn't avoid duplicates ... it was taking very long just to scrape a 100 distinct jobs ..

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

options = Options()
driver = webdriver.Chrome(options=options)
driver.set_window_size(1120, 1000)

url = "https://www.glassdoor.co.in/Job/data-science-jobs-SRCH_KO0,12.htm"
driver.get(url)  # opens up this link in new chrome window
jobs = []

# Set to track unique job IDs or identifiers
unique_jobs = set()

# Function to click "Show More" button if available
def click_show_more():
    try:
        # Wait for the "Show More" button and click
        show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb'))
        )
        show_more_button.click()
        time.sleep(2)  # Sleep to allow new content to load
    except Exception as e:
        print("Show More button not available or error:", e)

# Initial click to close any popup
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb'))
    ).click()
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
except Exception as e:
    print("Initial Show More or popup close failed:", e)

# Main loop for scraping pages
for j in range(60):  # Adjust the range based on how many pages you want to scrape
    click_show_more()
    time.sleep(2)  # Ensure the page is fully loaded before scraping
    
    job_buttons = driver.find_elements(By.CLASS_NAME, "JobsList_jobListItem__wjTHv")
    companies = driver.find_elements(By.CLASS_NAME, 'EmployerProfile_compactEmployerName__LE242')
    descriptions = driver.find_elements(By.CLASS_NAME, "JobCard_jobDescriptionSnippet__yWW8q")
    salaries = driver.find_elements(By.CLASS_NAME, 'JobCard_salaryEstimate__arV5J')
    
    for i, button in enumerate(job_buttons):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
        try:
            button.click()
            time.sleep(2)  # Allow time for the job details to load
        except Exception as e:
            print(f"Error clicking job button: {e}")
            continue
        
        try:
            company_name = companies[i].text
            #job_id = button.get_attribute('data-id')  # Hypothetical attribute, replace as necessary
            
            # Check if this job is already seen
            #if job_id in unique_jobs:
            #    continue
            
            # Add to unique set
            #unique_jobs.add(job_id)
            
            location = driver.find_element(By.CLASS_NAME, 'JobDetails_location__mSg5h').text
            job_title = driver.find_element(By.XPATH, '//div[@class="JobDetails_jobDetailsHeader__Hd9M3"]//h1').text
            job_description = descriptions[i].text
            
            salary_estimate = salaries[i].text if i < len(salaries) else -1
            other_details = driver.find_element(By.CLASS_NAME, "JobDetails_companyOverviewGrid__3t6b4").text if len(driver.find_elements(By.CLASS_NAME, "JobDetails_companyOverviewGrid__3t6b4")) > 0 else "N/A"
            rating = driver.find_element(By.CLASS_NAME, "RatingHeadline_sectionRatingScoreLeft__di1of").text if len(driver.find_elements(By.CLASS_NAME, "RatingHeadline_sectionRatingScoreLeft__di1of")) > 0 else "N/A"
            
            jobs.append({
                'company_name': company_name,
                'location': location,
                'job_title': job_title,
                'job_description': job_description,
                'salary_estimate': salary_estimate,
                'other_details': other_details,
                'rating': rating
            })
        except Exception as e:
            print(f"Error scraping job details: {e}")
    



Error clicking job button: Message: element not interactable
  (Session info: chrome=128.0.6613.86)
Stacktrace:
	GetHandleVerifier [0x00007FF62A6FB632+29090]
	(No symbol) [0x00007FF62A66E6E9]
	(No symbol) [0x00007FF62A52AFF9]
	(No symbol) [0x00007FF62A580432]
	(No symbol) [0x00007FF62A572D61]
	(No symbol) [0x00007FF62A5A66EA]
	(No symbol) [0x00007FF62A5726A6]
	(No symbol) [0x00007FF62A5A6900]
	(No symbol) [0x00007FF62A5C65D9]
	(No symbol) [0x00007FF62A5A6493]
	(No symbol) [0x00007FF62A5709B1]
	(No symbol) [0x00007FF62A571B11]
	GetHandleVerifier [0x00007FF62AA1883D+3294125]
	GetHandleVerifier [0x00007FF62AA64423+3604371]
	GetHandleVerifier [0x00007FF62AA5A2E7+3563095]
	GetHandleVerifier [0x00007FF62A7B6F16+797318]
	(No symbol) [0x00007FF62A67986F]
	(No symbol) [0x00007FF62A675454]
	(No symbol) [0x00007FF62A6755E0]
	(No symbol) [0x00007FF62A664A7F]
	BaseThreadInitThunk [0x00007FF9493E7374+20]
	RtlUserThreadStart [0x00007FF94AE3CC91+33]

Error clicking job button: Message: element not int

KeyboardInterrupt: 

In [17]:
# Convert to DataFrame and drop any residual duplicates
df = pd.DataFrame(jobs)
#df.drop_duplicates(inplace=True)

# Save the DataFrame
#df.to_csv('glassdoor_jobs.csv', index=False)

# Close the driver
driver.quit()

In [19]:
df.shape

(609, 7)

In [23]:
df.shape[0] - df.duplicated().sum()

263

In [30]:
df[df['salary_estimate'] == -1].shape[0] - df[df['salary_estimate'] == -1].duplicated().sum()

139

In [14]:
len(jobs)

609

In [21]:
len(job_buttons)#.text

211

In [16]:
click_show_more()

### All trial codes below to test different aspects of the code 

In [12]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [13]:
get_jobs("data scientist", 5, False)

AttributeError: 'WebDriver' object has no attribute 'find_element_by_class_name'

### Trial of different settings for the code 

In [5]:
webdriver.Chrome?

In [8]:
options = webdriver.ChromeOptions()

In [9]:
options

r"E:\Download\chromedriver-win64\chromedriver-win64",

In [13]:
# JUST A TRIAL CODE - to see if we are able to fetch elements

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = options)

#url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + "data scientist" + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
url = "https://www.glassdoor.co.in/Job/" + "data science" + "-SRCH_KO0,12.htm"
driver.get(url)

# element = WebDriverWait(driver,30).until(
# EC.presence_of_element_located((By.CLASS_NAME,'jl'))
# )


driver.find_element(By.CLASS_NAME,"JobsList_jobListItem__wjTHv")
#element = driver.find_element(By.XPATH, "//div[@class='jl']")

<selenium.webdriver.remote.webelement.WebElement (session="01fbfe764b2b3c9391ffa1344c4b1a46", element="f.73CB700E1A9FBB690958FF9BBEFD669F.d.7623E4C863F73C5EA4603475EB81E39A.e.52")>

driver.find_element(By.CLASS_NAME,"JobsList_dividerWithSelected__nlvH7") can't select element when specified with multiple class names i.e names separated by spaces in the inspect page , so give one class name at a time

In [22]:
# Just a trial CODE BLOCK - to understand the flow of the function and its different things

options = webdriver.ChromeOptions()

# not writing the executable parameter option since 
# it gives error now , maybe in updated versions
# of selenium
driver = webdriver.Chrome(options = options) 
driver.set_window_size(1120,1000)

url = "https://www.glassdoor.co.in/Job/" + "data-science-jobs" + "-SRCH_KO0,12.htm"
driver.get(url) #opens up this link in new chrome window
jobs = []

while( len(jobs) < 1):
    # time.sleep(slp_time)
    
    job_buttons = driver.find_elements(By.CLASS_NAME,"JobsList_jobListItem__wjTHv")
    
    for job in job_buttons:
        job.click()
        time.sleep(1)
        collected_successfully = False
        
        while not collected_successfully :
            try :
                company_name = driver.find_element(By.XPATH,'')
                location = driver.find_element(By.XPATH,'')
                job_title = driver.find_element(By.XPATH,'')
                job_description = driver.find_element(By.XPATH,'')
                collected_successfully = True
            except:
                time.sleep(5)
                
        break
    jobs.append(job_buttons)

### Handling sign up pop-up

In [17]:
# options = Options()
# options.add_argument("--disable-notifications")
# driver = webdriver.Chrome(options = options)
job_buttons[5].click()
#driver.find_element(By.XPATH,'//div[contains(@class,"ui-dialog") and @aria-describedby="dialogContent2"]//button[@title="Close"]').click()
#driver.find_element(By.CLASS_NAME,'TwoColumnLayout_backButton__kqDs_').click()
#driver.find_element(By.CLASS_NAME,'globalNavContainer__globalNavContainerStyles__mobileCenterLink').click()

# this below code finally helps get rid of the signup pop up
from selenium.webdriver.common.keys import Keys
driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)


In [3]:
# CURRENT CODE BLOCK 
# ERRORS ENCOUNTERD = 1) WRONG JOB TITLE FETCHED 2) JOB DESCRIPTION ELEMENT NOT FOUND

#options = webdriver.ChromeOptions()
options = Options()
options.add_argument("--disable-notifications") # did not do any good
driver = webdriver.Chrome(options = options)

# not writing the executable parameter in option since 
# it gives error now , maybe in updated versions
# of selenium

driver.set_window_size(1120,1000)

url = "https://www.glassdoor.co.in/Job/" + "data-science-jobs" + "-SRCH_KO0,12.htm"
driver.get(url) #opens up this link in new chrome window
jobs = []
num_jobs = 1
#while(len(jobs) < num_jobs):
try:
    driver.find_element(By.CLASS_NAME,"selected").click() #did not do any good
except :
    pass

time.sleep(.1)

try:
    close_button = driver.find_element(By.CSS_SELECTOR,'[alt="close"]').click() # did not do any good
    close_button.click()
    print('x out worked')
except NoSuchElementException:
    #print('x out did not work')
    pass


job_buttons = driver.find_elements(By.CLASS_NAME,"JobsList_jobListItem__wjTHv")
for job_button in job_buttons:
    print("Progress: {}".format(""+str(len(jobs))+ "/" + str(num_jobs)))
    if(len(jobs) >= num_jobs):
        break
    
    job_button.click()
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE) # this is what worked for sign-up prompt
    
    time.sleep(1)
    collected_successfully = False
        
    while not collected_successfully :
        try :
            company_name = driver.find_element(By.CLASS_NAME,'EmployerProfile_compactEmployerName__LE242').text
            location = driver.find_element(By.CLASS_NAME,'JobDetails_location__mSg5h').text
            job_title = driver.find_element(By.CLASS_NAME,'heading_Heading__BqX5J').text
            #job_description = driver.find_element(By.CLASS_NAME,'JobDetails_jobDescription__uW_fk').text
            collected_successfully = True
        except:
            time.sleep(5)
        
        #try:
        salaries = driver.find_elements(By.CLASS_NAME,'JobCard_salaryEstimate__arV5J')#.text
        salary_estimate = salaries[0].text

    jobs.append({"Job Title" : job_title,
            "Salary":salary_estimate,
            "Company Name" : company_name,
            "Location" : location,
            })

pd.DataFrame(jobs)

Progress: 0/1
Progress: 1/1


,Job Title,Salary,Company Name,Location
0,Frequently Asked Questions,₹4L - ₹7L (Glassdoor Est.),Procter & Gamble,Mumbai


In [23]:
company_name

'S&P Global'

In [24]:
location

'Hyderābād'

In [25]:
job_title

'Frequently Asked Questions'

In [9]:
type(job_buttons)

list

In [10]:
len(job_buttons)

31

In [40]:
job_buttons[0].text

'Procter & Gamble\n4.1\nSenior Manager – SAP Security Governance: Risk and Compliance Management\nMumbai\n₹4L - ₹7L (Glassdoor Est.)\nJust so that you know: We are an equal opportunity employer and value diversity at our company. We produce globally recognized brands, and we grow the best……\nSkills: SAP, SOX, Analysis skills, SAP S/4HANA, RMF\n30d+'

In [18]:
#job_buttons[2].click()
#close_button = driver.find_element(By.CSS_SELECTOR,'[alt="close"]').click()
#close_button = driver.find_element(By.CLASS_NAME,"JobDetails_actionWrapper__vrkVQ")
#close_button = driver.find_element(By.CLASS_NAME,"action-menu_ActionMenuTrigger__1QkiW")
#close_button.click()

### above pop up menu's class here in this cell
<button class="action-menu_ActionMenuTrigger__lQkiW" type="button" aria-live="polite" aria-label="More options" data-test="" id="radix-:rn:" aria-haspopup="menu" aria-expanded="false" data-state="closed"><svg width="24" height="24" viewBox="0 0 24 24" fill="currentColor" xmlns="http://www.w3.org/2000/svg" aria-live="polite" aria-hidden="false" class="icon_Icon__ptI3R"><path d="M6 12a2 2 0 1 1-4 0 2 2 0 0 1 4 0ZM14 12a2 2 0 1 1-4 0 2 2 0 0 1 4 0ZM20 14a2 2 0 1 0 0-4 2 2 0 0 0 0 4Z"></path></svg></button>

In [25]:
for i in job_buttons:
    print(i.text)

S&P Global
4.1
Senior MLOps Engineer
Hyderābād
₹6L - ₹8L (Glassdoor Est.)
In this role, you will play a pivotal role in implementing our machine learning operations, ensuring the seamless deployment, monitoring, and management of our……
Skills: CI/CD, Elasticsearch, Big data, NoSQL, Git
30d+
Procter & Gamble
4.1
Senior Manager – SAP Security Governance: Risk and Compliance Management
Mumbai
₹4L - ₹7L (Glassdoor Est.)
The Senior Manager’s role will ensure that the SAP Security Strategy and Risk and Compliance Management frameworks are always up-to-date, new threats are……
Skills: SAP, SOX, Analysis skills, SAP S/4HANA, RMF
30d+
RVR PRO
AI and Technology Research Assistant
Hyderābād
₹20T - ₹35T (Employer Est.)
Easy Apply
In this role, you will support our efforts in exploring emerging technologies, conducting in-depth research, and contributing to the development of new……
Skills: Data analysis skills, Research, Databases, Analysis skills, Machine learning
21d
Optum
Manager Data Scientist
B

 '//*[@id="left-column"]/div[1]/ul' xpath

<li class="JobsList_jobListItem__wjTHv JobsList_selected__nFuMW"

### Now trying other columns for scraping like ratings, company size, founded etc.

In [5]:
options = Options()
driver = webdriver.Chrome(options = options)
url = "https://www.glassdoor.co.in/Job/" + "data-science-jobs" + "-SRCH_KO0,12.htm"
driver.get(url)

element = driver.find_element(By.CLASS_NAME, "JobDetails_companyOverviewGrid__3t6b4")
element.text

'Base pay range\nThe minimum salary is ₹4L and the max salary is ₹7L.\n₹4L – ₹7L/yr (Glassdoor est.)\n₹6L\n/yr Median\nMumbai\nIf an employer includes a salary or salary range on their job, we display it as an "Employer Estimate". If a job has no salary data, Glassdoor displays a "Glassdoor Estimate" if available. To learn more about "Glassdoor Estimates," see our FAQ page.'

In [6]:
# other details of the company
driver.find_element(By.CLASS_NAME, "JobDetails_companyOverviewGrid__3t6b4").text

'Size\n10000+ Employees\nFounded\n1837\nType\nCompany - Public\nIndustry\nConsumer Product Manufacturing\nSector\nManufacturing\nRevenue\n$10+ billion (USD)'

In [7]:
# Rating
driver.find_element(By.CLASS_NAME, "RatingHeadline_sectionRatingScoreLeft__di1of").text

'4.1'

### Handling wrong Job name and description element not found

In [92]:
for company in range(len(companies)) :
    print(companies[-1].text)

Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group
Boston Consulting Group


In [85]:
time.sleep(5)

In [70]:
show_more = driver.find_element(By.CLASS_NAME, 'JobsList_buttonWrapper__ticwb')
if(show_more):
    print("hi")
    show_more.click()
    if(show_more):
        print(2)
        show_more.click()

else :
    print("no")
show_more.text

hi
2


''

In [71]:
companies[0].text

'Procter & Gamble'

In [73]:
descriptions[0].text

'The Senior Manager’s role will ensure that the SAP Security Strategy and Risk and Compliance Management frameworks are always up-to-date, new threats are……\nSkills: SAP, SOX, Analysis skills, SAP S/4HANA, RMF'

In [74]:
company_name, location, job_title, job_description

('Boston Consulting Group',
 'Gurgaon',
 'Data Scientist, India - BCG X',
 'As a Data Scientist at BCG, you will collaborate on challenging projects to increasing your understanding of complex business problems from diverse perspectives……\nSkills: Image processing, Azure, Big data, R, Google Cloud Platform')

In [104]:
len(job_buttons)

151

In [95]:
if(job_buttons[-1].text):
    print("hi")
else:
    print("bye")

bye


In [96]:
len(descriptions)

30

In [105]:
df = pd.DataFrame(jobs)#.shape


206

In [108]:
df.duplicated().sum()


206

In [109]:
325-206

119

In [106]:
df

,company_name,location,job_title,job_description
0,Procter & Gamble,Mumbai,Senior Manager – SAP Security Governance: Risk...,The Senior Manager’s role will ensure that the...
1,Atlassian,Bengaluru,"Software Engineer, Data Science","You'll be responsible for the design, build an..."
2,Boston Consulting Group,Gurgaon,"Data Analyst, Optimization, India - X Delivery",You will collaborate with case teams to gather...
3,Optum,Bengaluru,Manager Data Scientist,Comply with the terms and conditions of the em...
4,S&P Global,Gurgaon,Apprentice – Data Analyst,We are responsible for assuring that data impa...
...,...,...,...,...
320,C2Perform India,Delhi,Director of Development Operations,The role is dedicated to transforming the way ...
321,Boston Consulting Group,Bengaluru,"Data & Analytics Lead, Client Ops, India - X D...",This global business unit is an integral part ...
322,FedEx,Tamil Nadu,Data/Business Analyst,Analyze historical data to identify trends and...
323,BOEING,Bengaluru,Lead Data Scientist,Bachelor’s degree or higher in computer scienc...


In [107]:
df.shape

(325, 4)

### Trials below for fetching different elements

In [8]:
driver.find_element(By.XPATH, '//div[@class="JobDetails_jobDetailsHeader__Hd9M3"]').text

'Adobe\n4.2\nSenior Computer Scientist\nBengaluru'

In [10]:
driver.find_element(By.XPATH, '//div[@class="JobDetails_jobDetailsHeader__Hd9M3"]//h1').text

'Senior Computer Scientist'

class name tried for job name - [@class="heading_Heading__BqX5J"]

In [16]:
lst = driver.find_elements(By.CLASS_NAME,'EmployerProfile_compactEmployerName__LE242')#.text
for i in range(len(lst)):
    print(lst[i].text)


Atlassian
Procter & Gamble
Arccus Inc
Yantravision Software Pvt Ltd
Boston Consulting Group
Boston Consulting Group
S&P Global
ANZ Banking Group
Optum
ZF
Wisdom college for creativity & design
Adobe
Adobe
Ananta Consulting Group
Optum
Raaj Ventures
BOEING
Alignerr
BigSpire Software
Wahy Lab Solutions
Sagility
MSD
Proyouth.in
Ignitho
Medtek.AI
JPMorgan Chase & Co
FELIX HOSPITAL
American Express Global Business Travel
Imurgence
OpsFree


### We noted that , writing to find element by class name was just fwtching the first one when we wanted the company name , so we can use find_elements instead and run a loop to fetch the right company

In [18]:
driver.find_elements(By.CLASS_NAME,'JobDetails_location__mSg5h')#.text

[<selenium.webdriver.remote.webelement.WebElement (session="2d94f6e25a6d39a6cf8ef8a2041b5b39", element="f.E59392EEAC35C7C867733106858BAE28.d.465028068C75994439519A9BBF6B68A9.e.93")>]

In [19]:
driver.find_element(By.CLASS_NAME,'JobDetails_jobDescription__uW_fk')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".JobDetails_jobDescription__uW_fk"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6ADBF9642+30946]
	(No symbol) [0x00007FF6ADBAE3D9]
	(No symbol) [0x00007FF6ADAA6FDA]
	(No symbol) [0x00007FF6ADAF822C]
	(No symbol) [0x00007FF6ADAF850C]
	(No symbol) [0x00007FF6ADB3DCB7]
	(No symbol) [0x00007FF6ADB1CAAF]
	(No symbol) [0x00007FF6ADB3B041]
	(No symbol) [0x00007FF6ADB1C813]
	(No symbol) [0x00007FF6ADAEA6E5]
	(No symbol) [0x00007FF6ADAEB021]
	GetHandleVerifier [0x00007FF6ADD2F84D+1301229]
	GetHandleVerifier [0x00007FF6ADD3BDC7+1351783]
	GetHandleVerifier [0x00007FF6ADD32A13+1313971]
	GetHandleVerifier [0x00007FF6ADC2DD16+245686]
	(No symbol) [0x00007FF6ADBB759F]
	(No symbol) [0x00007FF6ADBB3814]
	(No sym

In [51]:
driver.find_element(By.CLASS_NAME,'JobDetails_showHidden__C_FOA')#.text

[<selenium.webdriver.remote.webelement.WebElement (session="2d94f6e25a6d39a6cf8ef8a2041b5b39", element="f.E59392EEAC35C7C867733106858BAE28.d.465028068C75994439519A9BBF6B68A9.e.143")>]

In [56]:
# below one is fetching the complete job details with everything from that right section
driver.find_element(By.CLASS_NAME,'JobDetails_jobDetailsSection__y_xa6').text

'About the Role:\nGrade Level (for internal use): 10\n\nThe Team: The Data Science/AI/ML team is a newly formed innovation and deployment team within S&P Global Ratings that will be responsible for building and executing a bold vision around using Machine Learning, Artificial Intelligence, Data Science, knowledge engineering, and human computer interfaces for augmenting various business processes.\nThe Impact: This role will have a significant impact on the success of our data science, AI, and Machine Learning projects ranging from choosing which projects should be undertaken, to delivering highest quality solution, ultimately enabling our business processes and products with AI and Machine Learning solutions.\n\nWe are seeking a senior MLOps Engineer to join our MLOps team within the Data Science COE at S&P Global. As a senior MLOps Engineer, you will contribute to the deployment, monitoring, and management of machine learning models and data pipelines. You will work with a peer group

### Remember that all the below elements are fetched from the left column container where the elements are present as a list, so using find_element will only return the first occurance of the element

### to get the list of all elements we would have to use find_elements()

In [15]:
# for fetching skills and description

detail = driver.find_element(By.CLASS_NAME,"JobCard_jobDescriptionSnippet__yWW8q")#.text.split("\n")[0]
#for j in i.text.split('\n') :
print(detail.text)
    #print(skills)

The Senior Manager’s role will ensure that the SAP Security Strategy and Risk and Compliance Management frameworks are always up-to-date, new threats are……
Skills: SAP, SOX, Analysis skills, SAP S/4HANA, RMF


In [36]:
# How to fetch skills
driver.find_elements(By.CLASS_NAME,"JobCard_jobDescriptionSnippet__yWW8q")[0].text.split('\n')[1]

'Skills: Software deployment, Microservices, SQL, Docker, Machine learning'

In [37]:
# how to fetch details of job
driver.find_elements(By.CLASS_NAME,"JobCard_jobDescriptionSnippet__yWW8q")[0].text.split('\n')[0]

"You'll be responsible for the design, build and maintenance of tools that enable our Support Engineers and related teams to be more effective and efficient at……"

In [43]:
# for fetching salary estimate
driver.find_element(By.CLASS_NAME,'JobCard_salaryEstimate__arV5J').text#.split()#[0]


'₹4L - ₹7L (Glassdoor Est.)'

In [48]:
# the paths specified in the video codes do not match the current 
# site's elements, below is an example to fetch job description 
#driver.find_element(By.XPATH,'.//div[@class="jobDescriptionContent"]').text